In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install geopandas

In [0]:
import json
from bokeh.io import show, output_file, save
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, NumeralTickFormatter,
                          GeoJSONDataSource, HoverTool,LogColorMapper,
                          LinearColorMapper, Slider, LogTicker)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure, save
import bokeh.plotting as bop

import numpy as np
import pandas as pd
from datetime import datetime

import geopandas as gpd


In [0]:
#!git clone https://github.com/nychealth/coronavirus-data.git

In [7]:
%cd drive/My\ Drive/DATATHON/

/content/drive/My Drive/DATATHON


In [8]:
file_ls = []
for k in range(12, 31):
  date = "0"+str(k) if len(str(k))==1 else str(k)
  file_nm = 'data/COVID_data/4_' + date +".csv"
  file_ls.append(file_nm)

for k in range(1, 13):
  date = "0"+str(k) if len(str(k))==1 else str(k)
  file_nm = 'data/COVID_data/5_' + date +".csv"
  file_ls.append(file_nm)
print(len(file_ls), file_ls)


covid_df_all = None

for i, file_path in enumerate(file_ls):
    covid_df = pd.read_csv(file_path)
    covid_df['MODZCTA'] = covid_df['MODZCTA'].astype('str').str.slice(0,-2)
    covid_df['date'] = i+1

    try:
      covid_df_all = pd.concat([covid_df_all, covid_df])
    except Exception:
      covid_df_all = covid_df.copy()
covid_df_all = covid_df_all.reset_index(drop=True)
covid_df_all.shape

31 ['data/COVID_data/4_12.csv', 'data/COVID_data/4_13.csv', 'data/COVID_data/4_14.csv', 'data/COVID_data/4_15.csv', 'data/COVID_data/4_16.csv', 'data/COVID_data/4_17.csv', 'data/COVID_data/4_18.csv', 'data/COVID_data/4_19.csv', 'data/COVID_data/4_20.csv', 'data/COVID_data/4_21.csv', 'data/COVID_data/4_22.csv', 'data/COVID_data/4_23.csv', 'data/COVID_data/4_24.csv', 'data/COVID_data/4_25.csv', 'data/COVID_data/4_26.csv', 'data/COVID_data/4_27.csv', 'data/COVID_data/4_28.csv', 'data/COVID_data/4_29.csv', 'data/COVID_data/4_30.csv', 'data/COVID_data/5_01.csv', 'data/COVID_data/5_02.csv', 'data/COVID_data/5_03.csv', 'data/COVID_data/5_04.csv', 'data/COVID_data/5_05.csv', 'data/COVID_data/5_06.csv', 'data/COVID_data/5_07.csv', 'data/COVID_data/5_08.csv', 'data/COVID_data/5_09.csv', 'data/COVID_data/5_10.csv', 'data/COVID_data/5_11.csv', 'data/COVID_data/5_12.csv']


(5518, 5)

In [9]:
# Read in shapefile and examine data
contiguous_nyc = gpd.read_file('coronavirus-data/Geography-resources/MODZCTA_2010.shp', SHAPE_RESTORE_SHX=True)
contiguous_nyc['MODZCTA'] = contiguous_nyc['MODZCTA'].astype('str')
contiguous_nyc.head(1)

,MODZCTA,label,geometry
0,10001,"10001, 10118","POLYGON ((987646.000 210360.000, 987522.000 21..."


In [0]:
def split_level(x):
  if x<478:
    return 0
  if x <818:
    return 1
  if x<1430:
    return 2
  else:
    return 3

data = pd.merge(contiguous_nyc, covid_df_all, how='left')
data['category'] = data.Positive.apply(lambda x: split_level(x))

In [0]:
# https://automating-gis-processes.github.io/CSC18/lessons/L5/advanced-bokeh.html
def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]

def getPolyCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

def getLineCoords(geometry, coord_type):
    """ Returns Coordinates of Linestring object."""
    return getXYCoords(geometry, coord_type)

def getPointCoords(geometry, coord_type):
    """ Returns Coordinates of Point object."""
    if coord_type == 'x':
        return geometry.x
    elif coord_type == 'y':
        return geometry.y

def multiGeomHandler(multi_geometry, coord_type, geom_type):
    """
    Function for handling multi-geometries. Can be MultiPoint, MultiLineString or MultiPolygon.
    Returns a list of coordinates where all parts of Multi-geometries are merged into a single list.
    """

    for i, part in enumerate(multi_geometry):
        # On the first part of the Multi-geometry initialize the coord_array (np.array)
        if i == 0:
            if geom_type == "MultiPoint":
                coord_arrays = np.append(getPointCoords(part, coord_type), np.nan)
            elif geom_type == "MultiLineString":
                coord_arrays = np.append(getLineCoords(part, coord_type), np.nan)
            elif geom_type == "MultiPolygon":
                coord_arrays = np.append(getPolyCoords(part, coord_type), np.nan)
        else:
            if geom_type == "MultiPoint":
                coord_arrays = np.concatenate([coord_arrays, np.append(getPointCoords(part, coord_type), np.nan)])
            elif geom_type == "MultiLineString":
                coord_arrays = np.concatenate([coord_arrays, np.append(getLineCoords(part, coord_type), np.nan)])
            elif geom_type == "MultiPolygon":
                coord_arrays = np.concatenate([coord_arrays, np.append(getPolyCoords(part, coord_type), np.nan)])

    # Return the coordinates
    return coord_arrays


def getCoords(row, geom_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon).
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geom_col]

    # Check the geometry type
    gtype = geom.geom_type

    # "Normal" geometries
    # -------------------

    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )

    # Multi geometries
    # ----------------

    else:
        return list( multiGeomHandler(geom, coord_type, gtype) )

In [0]:
data['x'] = data.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
data['y'] = data.apply(getCoords, geom_col="geometry", coord_type="y", axis=1)


sitesource = ColumnDataSource({'x': np.array(data['x'].apply(lambda x: np.array(x))), 
                               'y': np.array(data['y'].apply(lambda x: np.array(x))),
                               'category': data['category'].values,
                               'date': data['date'].values,
                               'zipcode': data['MODZCTA'].values,
                               'positive': data['Positive'].values})

In [0]:
# Make a slider object to toggle the day shown
slider = Slider(title = 'Date', 
                start = 1, end = 31, 
                step = 3, value = 1)

# This callback triggers the filter when the slider changes
callback = CustomJS(args = dict(source=sitesource), 
                    code = """source.change.emit();""")
slider.js_on_change('value', callback)
# Creates custom filter that selects the rows of the month based on the value in the slider
custom_filter = CustomJSFilter(args = dict(slider = slider, 
                                           source = sitesource), 
                               code = """
                                  var indices = [];
                                  // iterate through rows of data source and see if each satisfies some constraint
                                  for (var i = 0; i < source.get_length(); i++){
                                  if (source.data['date'][i] == slider.value){
                                  indices.push(true);
                                  } else {
                                  indices.push(false);
                                  }
                                  }
                                  return indices;
                                  """)
# Uses custom_filter to determine which set of sites are visible
view = CDSView(source = sitesource, filters = [custom_filter])

In [14]:
# Define color palettes
palette = brewer['Reds'][4] 
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 4)

 # Create color bar.
tick_labels = {'0':'','0.5':'23-478','1':'','1.5':'478-818',
               '2':'','2.5':'818-1430','3':'','3.5':'1430-4054',
                '4':''}
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 10,
                     width = 15, height = 250,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'vertical',
                     major_label_overrides = tick_labels)

# Create figure object.
p = bop.figure(title = 'NYC Covid Positive Cases 4/12 ~ 5/12', 
           plot_height = 500 ,
           plot_width = 550, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset, save")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('x','y', source = sitesource, view=view,
                   fill_color = {'field':'category',
                                 'transform':color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Zipcode','@zipcode'),
                                  ('Positive Cases','@positive')]))
p.add_layout(color_bar, 'right')
p = column(p, widgetbox(slider))

try:
    bop.reset_output()
    print("Reset")
    bop.output_notebook()
    print("NOTEBOOK")
    bop.show(p)  
    print("PLOT")
    
except:
    print("ERROR")
    bop.output_notebook()
    bop.show(p)  
    
    
  

Reset
NOTEBOOK


In [0]:

# Output filepath to HTML
output_file = r"./visualization/map_NYC.html"
try:
    bop.reset_output()
    print("Reset")
    bop.output_notebook()
    # Save the map
    bop.save(p, output_file)
    print("SAVE")
    
except:
    print("ERROR")
    bop.output_notebook()
    # Save the map
    bop.save(p, output_file)
    print("NOTEBOOK")
    
    
